In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [3]:
train

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
train.loc[train['Embarked'].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [6]:
train_clear = train.drop(['Cabin', 'Embarked', 'Name', 'PassengerId', 'Ticket'], axis=1)
test_clear = test.drop(['Cabin', 'Embarked', 'Name', 'PassengerId', 'Ticket'], axis=1)

In [7]:
# train_clear['Ticket'].unique().shape, train_clear['Ticket']

In [8]:
target = train_clear['Survived']
train_clear = train_clear.drop(['Survived'], axis=1)

In [9]:
train_cat = train_clear.select_dtypes(exclude=['number'])
train_cat = pd.concat((train_cat, train_clear['Pclass'].astype('string')), axis=1)

test_cat = test_clear.select_dtypes(exclude=['number'])
test_cat = pd.concat((test_cat, test_clear['Pclass'].astype('string')), axis=1)

cat_cols = train_cat.columns
num_cols = train_clear.drop(cat_cols, axis=1).columns

train_cat

,Sex,Pclass
0,male,3
1,female,1
2,female,3
3,female,1
4,male,3
...,...,...
886,male,2
887,female,1
888,female,3
889,male,1


In [10]:
cat_cols, num_cols

(Index(['Sex', 'Pclass'], dtype='object'),
 Index(['Age', 'SibSp', 'Parch', 'Fare'], dtype='object'))

In [11]:
train_cat_converted = pd.get_dummies(train_cat)
test_cat_converted = pd.get_dummies(test_cat)
train_cat_converted

,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
0,False,True,False,False,True
1,True,False,True,False,False
2,True,False,False,False,True
3,True,False,True,False,False
4,False,True,False,False,True
...,...,...,...,...,...
886,False,True,False,True,False
887,True,False,True,False,False
888,True,False,False,False,True
889,False,True,True,False,False


In [12]:
train[num_cols]

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Age,SibSp,Parch,Fare
0,22.0,1,0,7.2500
1,38.0,1,0,71.2833
2,26.0,0,0,7.9250
3,35.0,1,0,53.1000
4,35.0,0,0,8.0500
...,...,...,...,...
886,27.0,0,0,13.0000
887,19.0,0,0,30.0000
888,NaN,1,2,23.4500
889,26.0,0,0,30.0000


In [13]:
train_dummified = pd.concat([train_cat_converted, train[num_cols]], axis=1)
test_dummified = pd.concat([test_cat_converted, test[num_cols]], axis=1)
train_dummified

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Age,SibSp,Parch,Fare
0,False,True,False,False,True,22.0,1,0,7.2500
1,True,False,True,False,False,38.0,1,0,71.2833
2,True,False,False,False,True,26.0,0,0,7.9250
3,True,False,True,False,False,35.0,1,0,53.1000
4,False,True,False,False,True,35.0,0,0,8.0500
...,...,...,...,...,...,...,...,...,...
886,False,True,False,True,False,27.0,0,0,13.0000
887,True,False,True,False,False,19.0,0,0,30.0000
888,True,False,False,False,True,NaN,1,2,23.4500
889,False,True,True,False,False,26.0,0,0,30.0000


In [14]:
from sklearn.impute import KNNImputer
Imputer = KNNImputer(n_neighbors=5)
# imputed_train = Imputer.fit_transform(train_dummified)
# imputed_test = Imputer.fit_transform(test_dummified)

# imputed_train = pd.DataFrame(imputed_train, columns=train_dummified.columns)
# imputed_test = pd.DataFrame(Imputer.transform(test_dummified), columns=train_dummified.columns)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X, y = train_dummified, target
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# SVM

In [18]:
clf1 = make_pipeline(Imputer, StandardScaler(), SVC())
clf1.fit(X_train, y_train)

Pipeline(steps=[('knnimputer', KNNImputer()),
                ('standardscaler', StandardScaler()), ('svc', SVC())])

In [19]:
param_grid = {'svc__C':[0.1, 1, 10],
             'svc__kernel':['linear', 'poly', 'rbf', 'sigmoid'],
             'svc__gamma':['scale','auto']}
searcher = GridSearchCV(clf1, param_grid=param_grid, cv=5)
searcher.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('knnimputer', KNNImputer()),
                                       ('standardscaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'svc__C': [0.1, 1, 10],
                         'svc__gamma': ['scale', 'auto'],
                         'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid']})

In [20]:
searcher.best_score_

0.8229882793263075

In [21]:
pred = clf1.predict(X_val)
acc = accuracy_score(pred, y_val)
acc

0.8100558659217877

# Catboost

In [22]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(verbose=0)

In [23]:
cbclf = make_pipeline(Imputer, StandardScaler(), model)
cbclf.fit(X_train, y_train)

pred = cbclf.predict(X_val)
acc = accuracy_score(pred, y_val)
acc

0.8100558659217877

In [24]:
param_grid = {'catboostclassifier__learning_rate':(0.01, 0.05, 0.1, 0.5, 1),
              'catboostclassifier__n_estimators':(1000, 100),
              'catboostclassifier__depth':(1, 2, 3, 4, 5, 6),
              'catboostclassifier__verbose':[False]}
searcher = GridSearchCV(cbclf, param_grid=param_grid, cv=5, verbose=1)
searcher.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('knnimputer', KNNImputer()),
                                       ('standardscaler', StandardScaler()),
                                       ('catboostclassifier',
                                        <catboost.core.CatBoostClassifier object at 0x7d5bca4a7490>)]),
             param_grid={'catboostclassifier__depth': (1, 2, 3, 4, 5, 6),
                         'catboostclassifier__learning_rate': (0.01, 0.05, 0.1,
                                                               0.5, 1),
                         'catboostclassifier__n_estimators': (1000, 100),
                         'catboostclassifier__verbose': [False]},
             verbose=1)

In [25]:
searcher.best_score_

0.8314192849404117

In [26]:
pred = cbclf.predict(X_val)
acc = accuracy_score(pred, y_val)
acc

0.8100558659217877

In [27]:
test_dummified

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Age,SibSp,Parch,Fare
0,False,True,False,False,True,34.5,0,0,7.8292
1,True,False,False,False,True,47.0,1,0,7.0000
2,False,True,False,True,False,62.0,0,0,9.6875
3,False,True,False,False,True,27.0,0,0,8.6625
4,True,False,False,False,True,22.0,1,1,12.2875
...,...,...,...,...,...,...,...,...,...
413,False,True,False,False,True,NaN,0,0,8.0500
414,True,False,True,False,False,39.0,0,0,108.9000
415,False,True,False,False,True,38.5,0,0,7.2500
416,False,True,False,False,True,NaN,0,0,8.0500


In [28]:
test_pred = cbclf.predict(test_dummified)
predict = pd.concat((test['PassengerId'],
                     pd.DataFrame(test_pred, index=test_dummified.index, columns=['Survived'])),
                   axis=1, join='inner')[['PassengerId', 'Survived']]
predict.to_csv('catboost_predict.csv', index=False)
# output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': test_pred})
# output.to_csv('catboost_predict.csv')